# Testing database update code

In [1]:
#
from nrutils.core import global_settings
from os.path import dirname, basename, isdir, realpath, abspath, join, splitext, isfile
from os import pardir,system,popen
import pickle
import glob
from nrutils.core.basics import *
from nrutils.core.nrsc import scconfig, scentry
from commands import getstatusoutput as bash

(positive)>> Applying custom matplotlib settings.


In [2]:
# name of database to be updated
db_to_update = 'hogshead'

# path to the new simulation directory
new_sim_path = '/mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480'

In [5]:
# find the database location
cpath_list = glob.glob( global_settings.config_path+'*.ini' )

if isinstance(db_to_update,(str,unicode)):
    msg = 'Filtering ini files for \"%s\"'%cyan(db_to_update)
    alert(msg,'refresh_db')
    cpath_list = filter( lambda path: db_to_update in path, cpath_list )

(refresh_db)>> Filtering ini files for "hogshead"


In [7]:
# get the config information
config = scconfig(cpath_list[0])

# Create streaming log file
logfstr = global_settings.database_path + '/' + splitext(basename(config.config_file_location))[0] + '.log'
msg = 'Opening log file in: '+cyan(logfstr)
alert(msg)
logfid = open(logfstr, 'w')

# cull a possible list of databases to the first entry
catalog_dir = config.catalog_dir
if isinstance( config.catalog_dir, list ):
    warning('Multiple catalog directories found. We will scan through the related list, \
    and then store first the catalog_dir that the OS can find.')
    for d in config.catalog_dir:
        from os.path import isdir
        if isdir(d):
            catalog_dir = d
            warning('Selecting "%s"'%cyan(d))
            break

In [10]:
# path to current working database
current_database_path = global_settings.database_path + splitext(basename(config.config_file_location))[0] \
+ '.' + global_settings.database_ext

In [12]:
# open the database if it exists
if isfile(current_database_path):
    msg = 'Opening database file in: ' + cyan(current_database_path)
    alert(msg)
    with open( current_database_path, 'r') as data_file:
        current_database = pickle.load(data_file)
else:
    msg = "Database doesn't exist!"+cyan(current_database_path)
    error(msg)

(notebook)>> Opening database file in: /Users/jthompson/code/git_repos/nrutils_dev/nrutils/database/hogshead.db


In [13]:
# Generate a list of current simulation directories in the database
current_db_dirs = [e.raw_metadata.source_dir[0] for e in current_database ]
current_db_simnames = [e.simname for e in current_database ]

In [14]:
# check to see if the simulation is already in the database by directory information
if new_sim_path in current_db_dirs:
    msg = 'The "new" simulation is already in the database!'
    alert(msg)

In [15]:
# find the .bbh files that we want (or whatever file the particular database cares about)
msg = 'Searching for %s in %s.' % ( cyan(config.metadata_id), cyan(new_sim_path) ) + \
yellow(' This may take a long time if the folder being searched is mounted from a remote drive.')
alert(msg)
mdfile_list = rfind(new_sim_path,config.metadata_id,verbose=True)
alert('done.')

(notebook)>> Searching for .bbh in /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480. This may take a long time if the folder being searched is mounted from a remote drive.
(rfind)>> Seaching for .bbh in /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480:
  ->  /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480/psi4modes.bbh
  ->  /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480/q4a04t60dPm1_T_120_480.raw.bbh
  ->  /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480/q4a04t60dPm1_T_120_480.bbh
(notebook)>> done.


In [17]:
# update the database!
for mdfile in mdfile_list:
    # Create scentry object
    entry = scentry(config,mdfile,verbose=True)

    # write to log file
    logfid.write( '%5i\t%s\n'% (0,entry.log) )

    if entry.isvalid:
        # entry is valid and now we can check if it's already in the database via simname
        if entry.simname in current_db_simnames:
            msg = 'The "new" simulation name is already in the database!'
            alert(msg)
            break
            # quit()
        else:
            # If the obj is valid, add it to the catalog list
            alert('Simulation missing in the database and will be added\t\t\t\t %s'%yellow(entry.simname))

            # Backup old database
            db_backup_path = current_database_path + '.backup'
            msg = 'Creating backup of database file to %s'%cyan(db_backup_path)
            alert(msg,'scbuild')
            with open(db_backup_path, 'wb') as dbf:
              pickle.dump( current_database , dbf, pickle.HIGHEST_PROTOCOL )

            # Append new entries to the old database
            new_database = current_database[:]
            new_database.append(entry)

            # Write updated database to file
            msg = 'Saving updated database file to %s'%cyan(current_database_path)
            alert(msg,'scbuild')
            with open( current_database_path, 'wb') as data_file:
                pickle.dump( new_database, data_file, pickle.HIGHEST_PROTOCOL )
    else:
        # .bbh file not valid to create scentry object
        msg = 'This entry is not valid: {}'.format(mdfile)
        alert(msg)

## The following is invalid: /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480/psi4modes.bbh
(notebook)>> This entry is not valid: /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480/psi4modes.bbh
## The following is invalid: /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480/q4a04t60dPm1_T_120_480.raw.bbh
(notebook)>> This entry is not valid: /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480/q4a04t60dPm1_T_120_480.raw.bbh
## Working: /mnt/hogshead/NR_data/ReducedData-hogshead/q4/120_point/q4a04t60dPm1_T_120_480/q4a04t60dPm1_T_120_480.bbh
(bam.py)>> Initial parameters corresponding to the bbh file's aftrejunktime will be used to populate metadata.
(scentry.learn_metadata)>> The handler is found to have a "infer_default_level_and_extraction_parameter" method. Rather than the config file, this method will be used to determine the default extraction parameter and level.
(notebo

In [ ]:
# Close the log file
logfid.close()